In [ ]:
import numpy
import networkx as nx
import random

def hamming_dist(n1, n2): 
    x = n1 ^ n2  
    set_bits = 0

    while (x > 0) : 
        set_bits += x & 1
        x >>= 1
    return set_bits

def get_rough_mt_fuji(optimal_genotype, num_bits, cost_of_mismatch, noise_dist, **noise_dist_args):
    #print(noise_dist(**noise_dist_args))
    genotype_graph = nx.Graph(optimal_genotype=optimal_genotype, num_bits=num_bits)
    fitnesses = numpy.zeros(2**num_bits)
    for i in range(0, 2**num_bits):
        distance_to_optimal = hamming_dist(i, optimal_genotype)
        distance_to_unity = hamming_dist(i, 2**num_bits-1)
        fitness = max(1 - cost_of_mismatch * distance_to_optimal + noise_dist(**noise_dist_args), 0.0001)
        

        genotype_graph.add_node(i, 
                                abundance=0,
                                fitness=fitness,
                                genotype_int=i, 
                                distance_to_opt=distance_to_optimal,
                                distance_to_unity=distance_to_unity)

        fitnesses[i] = fitness

    for i in range(0, 2**num_bits):
        for j in range(i, 2**num_bits):
            if hamming_dist(i, j) == 1:  
                genotype_graph.add_edge(i, j)

    return genotype_graph, fitnesses


class fuji:
    def __init__(self, optimal_genotype, n):
        graph, fitnesses = get_rough_mt_fuji(optimal_genotype, 10, .01, numpy.random.exponential, scale=0.0001, size=None)
        self.n = n
        self.fitnesses = fitnesses

    def fitness(self, genotype):
        return self.fitnesses[genotype]



In [ ]:
from fujilandscape import fuji
from random import randint
from backports.functools_lru_cache import lru_cache
import numpy


def binary(num, length):
    return format(num, '#0{}b'.format(length + 2))


class switching_class:
    fujilandscape1 = fuji(optimal_genotype=0, n=4)
    fujilandscape2 = fuji(optimal_genotype=15, n=4)

    def __init__(self, n=10):
        self.n = n
        self.bitstring = randint(0,(2**n) - 1)
        self.__current_fuji__ = self.fujilandscape1

    def __fitness__(self):
        return self.__current_fuji__.fitness(self.bitstring)

    def __mutate__(self):
        bstr = binary(self.bitstring, self.n)[2:]
        for _ in range(1):
            bstr = list(bstr)
            flip_index = randint(0, self.fujilandscape1.n - 1)
            bstr[flip_index] = '0' if bstr[flip_index] == '1' else '1'
            bstr = ''.join(bstr)
        self.bitstring = int(bstr, 2)
    
    @lru_cache(maxsize=64)
    def __mutational_neighborhood__(self):
        neighborhood = []
        for i in range(self.n):
            neighbor = self.__IndiGrow__.genotype_deepcopy(self)
            bstr = list(binary(neighbor.bitstring, self.n)[2:])
            bstr[i] = '0' if bstr[i] == '1' else '1'
            bstr = ''.join(bstr)
            neighbor.bitstring = int(bstr, 2)
            neighborhood.append(neighbor)
        return neighborhood

In [ ]:
class switching_event:
    def __init__(self):
        self.__IndiGrow__ = None
        self.state = 1
    
    def __run_handler__(self):
        for node in self.__IndiGrow__.population.vs():
            if self.state == 1:
                node['genotype'].__current_fuji__ = node['genotype'].fujilandscape2
            else:
                node['genotype'].__current_fuji__ = node['genotype'].fujilandscape1
        self.state = ~self.state + 2
        self.__IndiGrow__.mark_as_dirty(all_dirty=True)

class print_event:
    def __init__(self):
        self.__IndiGrow__ = None
    
    def __run_handler__(self):
        print(sum((self.__IndiGrow__.population.vs['fitness']))/len(self.__IndiGrow__.population.vs()))


In [ ]:
from IndiGrow.IndiGrow import IndiGrow
import random
import numpy

random.seed(182)
numpy.random.seed(1277)

population_size = 10E7
mutation_rate = 10E-4

test = IndiGrow(population_size=10E7, mutation_rate=mutation_rate)

test.add_genotype(genotype=switching_class(n=4), frequency=1)
test.add_event(event_class=switching_event, trigger_every=10)
test.add_event(event_class=print_event, trigger_every=1)




for i in range(300):
    test.timestep()
